In [1]:
from scripts.segment import *
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from functools import partial
from multiprocessing import Pool

In [2]:
# grid_size = 10
# num_simulation = 10
# num_test = 20
# fail_rates = np.linspace(0, 1, num=num_test)



# pool = Pool(processes=num_test)


In [3]:
def generate_sim_report_topo(grid_size,num_simulation,fail_rate):
    report = SegmentReport(grid_size)
    valves = generate_valves_grid(grid_size)
    valve_register = ValveRegister()
    register_valves(valves,valve_register)
    for i in range(num_simulation):
        valves_dict = generate_valves_dict(valve_register,x,fail_rate)
        segments = simulate_segments(grid_size,valves_dict)
        report = update_segment_report(segments,report)
    return report

def generate_nx_config(valve_register,x):
    nid2valve = valve_register.nid2v
    vids2fail = []
    for _,valves in nid2valve.items():
        removed_valves = np.random.choice(valves, x, replace=False)
        for removed_valve in removed_valves:
            vids2fail.append(removed_valve.vid)
    
    vids = valve_register.vid2v.keys()
    remained_vids = list(set(vids)-set(vids2fail))

    return vids2fail,remained_vids

def generate_vids2fail(vids,fail_rate):
    num_failed_valves = int(fail_rate*len(vids))
    rand_fvids = list(np.random.choice(vids, num_failed_valves, replace=False))
    return rand_fvids

In [35]:
from scripts.node import *

In [36]:

        
grid_size = 3
valves = generate_valves_grid(grid_size)
valve_register = ValveRegister()
register_valves(valves,valve_register)

nids2node = get_nid2nodes(grid_size,valve_register.nid2v)
node = nids2node[0]

print (node)

nid: 0, ntype: NodeType.UPPER_LEFT,        nneighbors: left: None, right: 1, up None, down 3,        pneighbors: [None, 0, None, 6],        vneighbors: left: None, right: valve with vid 0, links to nid 0 and pid 0, fail status 0 , up: None, down: valve with vid 12, links to nid 0 and pid 6, fail status 0 ,        degree: 2


In [37]:
from scripts.topology_operations import *

In [30]:
set(range(10))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [34]:
mesh.degree_distribution

{0: [],
 1: [],
 2: [0, 4, 20, 24],
 3: [1, 2, 3, 5, 9, 15, 21, 22, 23],
 4: [12, 16, 17, 18, 14],
 5: [10],
 6: [13, 6],
 7: [],
 8: [],
 9: []}

In [43]:
from scripts.graph import * 

In [44]:
# def check_left_right(left_node,right_node):
#     left_node_valves = left_node.valve_neighbors
#     right_node_valves = right_node.valve_neighbors
#     if (not left_node_valves.right.fail) and (not right_node_valves.left.fail):
#         return True
#     return False

# def check_up_down(up_node,down_node):
#     up_node_valves = up_node.valve_neighbors
#     down_node_valves = down_node.valve_neighbors
#     if (not up_node_valves.down.fail) and (not down_node_valves.up.fail):
#         return True
#     return False
        
# def check_direct_linkage(node1,node2):
#     n1_neighbors = node1.get_node_neighbor_ids()
#     nid2 = node2.nid
#     if nid2 not in n1_neighbors:
#         return False
    
#     if n1_neighbors[0] == nid2:
#         return check_left_right(node2,node1)
#     if n1_neighbors[1] == nid2:
#         return check_left_right(node1,node2)
#     if n1_neighbors[2] == nid2:
#         return check_up_down(node2,node1)
#     if n1_neighbors[3] == nid2:
#         return check_up_down(node1,node2)
    
        
# grid_size = 3
# valves = generate_valves_grid(grid_size)
# valve_register = ValveRegister()
# register_valves(valves,valve_register)
# nids2node = get_nid2nodes(grid_size,valve_register.nid2v)

# check_direct_linkage(nids2node[1],nids2node[0])

True

In [45]:
# def get_direct_link_nodes(node,nids2node):
#     neighbors = node.get_node_neighbor_ids()
#     linked_nodes = []
#     for nid in neighbors:
#         if (nid!=None):
#             neighbor_node = nids2node[nid]
#             if check_direct_linkage(node,neighbor_node):
#                 linked_nodes.append(neighbor_node)
#     return linked_nodes
        
# nodes = get_direct_link_nodes(nids2node[3],nids2node)
# for node in nodes:
#     print (node)

# # nodes = get_direct_link_nodes(nids2node[4],nids2node)
# # for node in nodes:
# #     print (node)

nid: 4, ntype: NodeType.NORMAL,        nneighbors: left: 3, right: 5, up 1, down 7,        pneighbors: [2, 3, 8, 9],        vneighbors: left: valve with vid 5, links to nid 4 and pid 2, fail status 0 , right: valve with vid 6, links to nid 4 and pid 3, fail status 0 , up: valve with vid 17, links to nid 4 and pid 8, fail status 0 , down: valve with vid 18, links to nid 4 and pid 9, fail status 0 ,        degree: 4
nid: 0, ntype: NodeType.UPPER_LEFT,        nneighbors: left: None, right: 1, up None, down 3,        pneighbors: [None, 0, None, 6],        vneighbors: left: None, right: valve with vid 0, links to nid 0 and pid 0, fail status 0 , up: None, down: valve with vid 12, links to nid 0 and pid 6, fail status 0 ,        degree: 2
nid: 6, ntype: NodeType.LOWER_LEFT,        nneighbors: left: None, right: 7, up 3, down None,        pneighbors: [None, 4, 7, None],        vneighbors: left: None, right: valve with vid 8, links to nid 6 and pid 4, fail status 0 , up: valve with vid 15, lin

In [46]:
# def assemble_adjacency_mtx_nodes(nids2node):
#     tot_nodes = len(nids2node)
#     rows,cols = [],[]
#     for nid,node in nids2node.items():
#         connected_nodes = get_direct_link_nodes(node,nids2node)
#         for connected_node in connected_nodes:
#             rows.append(nid)
#             cols.append(connected_node.nid)

#     vals = 1+np.zeros(len(rows))
#     A = csc_matrix((vals, (rows, cols)), shape=(tot_nodes,tot_nodes))
#     return A

# A = assemble_adjacency_mtx_nodes(nids2node)
# print (A.todense())

[[0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0.]]


In [47]:
def create_reversed_valves(valves):
    reversed_valves = []
    for v in valves:
        valve = v.copy()
        valve.fail = ~valve.fail
        reversed_valves.append(valve)
    return reversed_valves

In [48]:
grid_size = 3
valves = generate_valves_grid(grid_size)
valve_register = ValveRegister()
register_valves(valves,valve_register)
nids2node = get_nid2nodes(grid_size,valve_register.nid2v)
len(nids2node)
# reversed_valve_dict = reverse_valve_fail(valve_register.vid2v)
# nids2node = get_nid2nodes(grid_size,valve_register.vid2v)
# A = assemble_adjacency_mtx(grid_size,reversed_valve_dict)

9

In [69]:
# def node_pair2pid(node1,node2):
#     pids1 = set(node1.get_pipe_neighbor_ids())
#     pids2 = set(node2.get_pipe_neighbor_ids())
#     pids = pids1.intersection(pids2)
    
#     pid = None 
#     for p in pids:
#         if p!= None:
#             pid = p
#     return pid
 
# node1 = nids2node[1]
# node2 = nids2node[4]

# node_pair2pid(node1,node2)

8

In [70]:
# def convert_npairs2pids(node_pairs,nids2node):
#     pids = []
#     for nid1,nid2  in node_pairs:
#         n1,n2 = nids2node[nid1],nids2node[nid2]
#         pids.append(node_pair2pid(n1,n2))
#     return pids
        
        

    
    

In [127]:
# def get_bfs_edges(A,nids2node):
#     component,edges = bfs_tree(0,A)
#     edges = convert_npairs2pids(edges,nids2node) 
#     return edges

get_bfs_edges(A,nids2node)

[6, 7, 4, 5, 11, 3, 8, 1]

In [166]:
# class Node(object):
#     def __init__(self,nid,node_type_checker,grid_size):
#         self.nid = nid
#         self.node_type = node_type_checker.get_node_type(nid)
#         self.node_neighbors = NodeNeighbors(nid,grid_size)
#         self.valve_neighbors = ValveNeighbors(nid)
#         self.pipe_neighbors = [None,None,None,None]
#         self.closed_pipes = [False,False,False,False]
    
#     def adjust_neighbors(self,nid2valve):
#         self.valve_neighbors.adjust_neighbors(nid2valve,self.node_type)
#         self.node_neighbors.adjust_neighbors(self.valve_neighbors)
#         self.pipe_neighbors = self.create_pipe_neighbors()
    
#     def create_pipe_neighbors(self):
#         pipe_neighbors = [None,None,None,None]
#         v_neighbors = self.get_valve_neighbors()
#         for i,v in enumerate(v_neighbors):
#             if v:
#                 pipe_neighbors[i] = v.pid
#             else:
#                 self.closed_pipes[i] = True
#         return pipe_neighbors
    
#     def get_node_neighbor_ids(self):
#         return self.node_neighbors.get_neighbors()
    
#     def get_valve_neighbors(self):
#         return self.valve_neighbors.get_neighbors()
    
#     def get_pipe_neighbor_ids(self):
#         return self.pipe_neighbors
    
#     def degree(self):
#         d = 4-sum(self.closed_pipes)
#         return d
        
        
#     def __str__(self):  
#         return f"nid: {self.nid}, ntype: {self.node_type},\
#         nneighbors: {self.node_neighbors},\
#         pneighbors: {self.pipe_neighbors},\
#         vneighbors: {self.valve_neighbors},\
#         degree: {self.degree()}" 

    
# def get_nid2nodes(grid_size,nid2v_dict):
#     nid2nodes = {}
#     type_checker = NodeTypeChecker(grid_size)
#     N = grid_size*grid_size
#     for nid in range(N):
#         node = Node(nid,type_checker,grid_size)
#         node.adjust_neighbors(nid2v_dict)
#         nid2nodes[nid] = node     
#     return nid2nodes 

In [167]:
# class TestNodes(unittest.TestCase):
    
#     def setUp(self):
#         grid_size = 3
#         valves = generate_valves_grid(grid_size)
#         valve_register = ValveRegister()
#         register_valves(valves,valve_register)
#         self.nids2node = get_nid2nodes(grid_size,valve_register.nid2v)
    
#     def test_center_node(self):
#         nid = 4
#         node = self.nids2node[nid]
#         """Nodes"""
#         self.assertEqual(node.get_node_neighbor_ids(),[3,5,1,7])
#         """valves """
#         vids = []
#         for v in node.get_valve_neighbors():
#             vids.append(v.vid)
            
#         self.assertEqual(vids,[5,6,17,18])
#         """pipes"""
#         self.assertEqual(node.get_pipe_neighbor_ids(),[2,3,8,9])
#         """degree"""
#         self.assertEqual(node.degree(),4)
#         # Closed pipes 
#         self.assertEqual(node.closed_pipes,[False,False,False,False])
    
#     def test_corner_node(self):
#         nid = 0
#         node = self.nids2node[nid]
#         """Nodes"""
#         self.assertEqual(node.get_node_neighbor_ids(),[None,1,None,3])
#         """valves """
#         vids = []
#         for v in node.get_valve_neighbors():
#             try:
#                 vids.append(v.vid)
#             except:
#                 vids.append(None)
            
#         self.assertEqual(vids,[None,0,None,12])
#         """pipes"""
#         self.assertEqual(node.get_pipe_neighbor_ids(),[None,0,None,6])
#         """degree"""
#         self.assertEqual(node.degree(),2)
#         # Closed pipes 
#         self.assertEqual(node.closed_pipes,[True,False,True,False])
        

        
# unittest.main(argv=['first-arg-is-ignored'], exit=False)
        

..
----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


In [275]:
# class Mesh(object):
#     def __init__(self,grid_size):
#         self.grid_size = grid_size
#         self.nids = set(range(grid_size*grid_size))
        
#         self.valve_register = self._create_valveNregister()
#         self.nid2nodes = self._create_nid2nodes()
#         self.contraction_reg = ContractionRegister()
#         self._init_degree_distribution()
        
#         self.adj_mtx_nodes = assemble_adjacency_mtx_nodes(self.nid2nodes)
#         self.bfs_edges = get_bfs_edges(self.adj_mtx_nodes,self.nid2nodes)
#         self.closed_pids = set()
# #         self.deletion_reg = DeletionRegister()
        
#     def _create_valveNregister(self):
#         valves = generate_valves_grid(self.grid_size)
#         valve_register = ValveRegister()
#         register_valves(valves,valve_register)
#         return valve_register
    
#     def _create_nid2nodes(self):
#         return get_nid2nodes(self.grid_size,self.valve_register.nid2v)
    
#     def _init_degree_distribution(self):
#         self.degree_distribution = {}
#         for i in range(10):
#             self.degree_distribution[i] = []
    
#     def update_degree_distribution(self):    
#         self._init_degree_distribution()
#         normal_nids = self.nids-self.contraction_reg.contracted_nids
#         for normal_nid in normal_nids:
#             node = self.nid2nodes[normal_nid]
#             degree = node.degree()
#             self.degree_distribution[degree].append(normal_nid)
        
#         for cn1,cn2 in self.contraction_reg.npairs:
#             degree = cn1.degree()+cn2.degree()-2
#             self.degree_distribution[degree].append(cn1.nid)
    
#     def perform_contractions(self,identify_pairs_op):
#         self.contraction_reg = register_contractions(identify_pairs_op,self.contraction_reg)
    
#     def reduce_degree_num(self,degree,desired_num,max_steps = 10):
#         self.update_degree_distribution()
#         nids = self.degree_distribution[degree]
#         step = 0
#         while (len(nids) > desired_num):
#             if step > max_steps: 
#                 raise Exception(f"Fail to reduce number of {degree} to {desired_num}")
#             nid2close = np.random.choice(nids,1)[0]
#             node2close = self.nid2nodes[nid2close]
#             pid2close = choose_pid2close(node2close,self.bfs_edges)
#             if pid2close != None:
#                 self.close_pipe_on_node(nid2close,pid2close)
#                 self.closed_pids.add(pid2close)
                
#             nids = self.degree_distribution[degree]
#             step+=1
    
#     def close_pipe_on_node(self,nid,pid2close):
#         node1 = self.nid2nodes[nid]
#         op = None
#         for i, pid in enumerate(node1.pipe_neighbors):
#             if pid ==  pid2close:
#                 node1.closed_pipes[i] = True
#                 op = i
#                 break
#         if op == None:
#             raise Exception(f"can not close pipe {pid2close}, not linked to this node")
            
#         nid2 = node1.get_node_neighbor_ids()[op]
#         node2 = self.nid2nodes[nid2]
#         self._close_pipe_on_other_end(node2,op)
#         self.update_degree_distribution()
            
#     def _close_pipe_on_other_end(self,node2,op):
#         if op == 0:
#             node2.closed_pipes[1] = True
#         elif op == 1:
#             node2.closed_pipes[0] = True
#         elif op == 2:
#             node2.closed_pipes[3] = True
#         elif op == 3:
#             node2.closed_pipes[2] = True
        
        

In [285]:
# class TestMesh(unittest.TestCase):
    
#     def setUp(self):
#         grid_size = 3
#         self.mesh = Mesh(grid_size)
        
        
#     def test_contraction(self):
#         n1,n2 = self.mesh.nid2nodes[3],self.mesh.nid2nodes[6]
#         identify_pairs_op = [[n1,n2,3]]
#         self.mesh.perform_contractions(identify_pairs_op)
#         self.mesh.update_degree_distribution()
#         print ("k dist after contraction", self.mesh.degree_distribution)
#         print ("contraction reg", self.mesh.contraction_reg)
#         self.assertEqual(len(self.mesh.degree_distribution[3]),4)
#         self.assertEqual(len(self.mesh.degree_distribution[2]),3)
        
#     def test_close_pipe_on_node(self):
#         nid = 1
#         pid2close = 8
#         self.mesh.close_pipe_on_node(nid,pid2close)
        
#         self.assertEqual(self.mesh.nid2nodes[1].closed_pipes, [False,False,True,True])
#         self.assertEqual(self.mesh.nid2nodes[4].closed_pipes, [False,False,True,False])
        
#         degree_dist = self.mesh.degree_distribution
#         self.assertEqual(len(degree_dist[4]),0)
#         self.assertEqual(len(degree_dist[3]),4)
#         self.assertEqual(len(degree_dist[2]),5)
#         print ("k dist after deletion ", self.mesh.degree_distribution)
        
#     def test_reduce_degree(self):
#         k = 4
#         num = 0
#         self.mesh.reduce_degree_num(k,num)
#         degree_dist = self.mesh.degree_distribution
#         closed_pids = self.mesh.closed_pids
#         self.assertEqual(len(degree_dist[k]),num)
#         self.assertEqual(len(closed_pids),1)
#         print ("k dist after reduction ", self.mesh.degree_distribution)
        
# unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....

k dist after deletion  {0: [], 1: [], 2: [0, 1, 2, 6, 8], 3: [3, 4, 5, 7], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}
k dist after contraction {0: [], 1: [], 2: [0, 2, 8], 3: [1, 5, 7, 3], 4: [4], 5: [], 6: [], 7: [], 8: [], 9: []}
contraction reg npairs: [(<__main__.Node object at 0x7fc125342518>, <__main__.Node object at 0x7fc125342358>)], vpairs: [(<scripts.valves.Valve object at 0x7fc125333cc0>, <scripts.valves.Valve object at 0x7fc125333c88>)],        trivial_pids: [7],contracted_nids: {3, 6}
k dist after reduction  {0: [], 1: [], 2: [0, 2, 6, 7, 8], 3: [1, 3, 4, 5], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}



----------------------------------------------------------------------
Ran 5 tests in 0.021s

OK


In [281]:
grid_size = 5
contraction_ratio = 0.2
mesh = Mesh(grid_size)

nids2identify = get_nids2identify(grid_size,contraction_ratio)
identify_pairs_op =  get_identify_pairs_op(nids2identify,mesh.nid2nodes)
mesh.perform_contractions(identify_pairs_op)
mesh.update_degree_distribution()
print(mesh.contraction_reg)

npairs: [(<__main__.Node object at 0x7fc1253594a8>, <__main__.Node object at 0x7fc125359c50>), (<__main__.Node object at 0x7fc12535bef0>, <__main__.Node object at 0x7fc12535b828>), (<__main__.Node object at 0x7fc1253f3208>, <__main__.Node object at 0x7fc1253f3a90>), (<__main__.Node object at 0x7fc12535b4a8>, <__main__.Node object at 0x7fc12535b240>)], vpairs: [(<scripts.valves.Valve object at 0x7fc125373eb8>, <scripts.valves.Valve object at 0x7fc125373358>), (<scripts.valves.Valve object at 0x7fc125368d30>, <scripts.valves.Valve object at 0x7fc125368b38>), (<scripts.valves.Valve object at 0x7fc125368c18>, <scripts.valves.Valve object at 0x7fc125368be0>), (<scripts.valves.Valve object at 0x7fc125368908>, <scripts.valves.Valve object at 0x7fc125368400>)],        trivial_pids: [0, 26, 19, 22],contracted_nids: {0, 1, 10, 11, 15, 16, 23, 24}


In [130]:
mesh.degree_distribution

{0: [],
 1: [],
 2: [0, 4, 24],
 3: [1, 2, 3, 5, 9, 10, 19, 21, 20],
 4: [6, 8, 11, 16, 17, 18, 23],
 5: [14],
 6: [7],
 7: [],
 8: [],
 9: []}

In [176]:
print(identify_pairs_op[0][1])

nid: 6, ntype: NodeType.LOWER_LEFT,        nneighbors: left: None, right: 7, up 3, down None,        pneighbors: [None, 4, 7, None],        vneighbors: left: None, right: valve with vid 8, links to nid 6 and pid 4, fail status 0 , up: valve with vid 15, links to nid 6 and pid 7, fail status 0 , down: None,        degree: 2


..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


In [153]:
# def choose_pid2close(node,pid_blacklist):
#     valid_pids = []
#     pids = [pid for pid in node.pipe_neighbors if (pid != None and pid not in pid_blacklist) ]
#     if len(pids):
#         return np.random.choice(pids,1)[0]
#     else:
#         return None
    

In [154]:
grid_size = 3
contraction_ratio = 0.2
mesh = Mesh(grid_size)

nids2identify = get_nids2identify(grid_size,contraction_ratio)
identify_pairs_op =  get_identify_pairs_op(nids2identify,mesh.nid2nodes)
mesh.perform_contractions(identify_pairs_op)
mesh.update_degree_distribution()
print(mesh.contraction_reg)

npairs: [(<__main__.Node object at 0x7fc125d52d30>, <__main__.Node object at 0x7fc125ade2e8>)], vpairs: [(<scripts.valves.Valve object at 0x7fc125c99da0>, <scripts.valves.Valve object at 0x7fc125c991d0>)],        trivial_pids: [7],contracted_nids: {3, 6}


In [155]:
nid = 4
node = mesh.nid2nodes[nid]
pid2close = choose_pid2close(node,mesh.bfs_edges)
node.close_pipe(pid2close)


TypeError: 'method' object is not subscriptable

In [120]:
len(mesh.bfs_edges)

12